In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("checkins.dat",  skipinitialspace = True, sep='|', nrows=10 ** 5).dropna()
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


In [2]:
coord = data[['latitude      ','longitude     ']]

In [3]:
from sklearn.cluster import MeanShift

msh = MeanShift(bandwidth=0.1)
cl_labels = msh.fit_predict(coord)

In [4]:
sign_clusters = []

for el in cl_labels:
    if el not in sign_clusters and np.count_nonzero(cl_labels == el) > 15:
        sign_clusters.append(el)
        
sign_clusters.sort()

In [5]:
coord['cluster'] = cl_labels
# coord = coord.reset_index(drop=True)
coord = coord.as_matrix() # x, y, cluster num

In [6]:
cl_sum = {}

for el in sign_clusters:
    cl_sum[el] = [0,0,0] # x_sum, y_sum, num of points
    
for str in coord:
    if str[2] in sign_clusters:
        cl_sum[str[2]][0] += str[0]
        cl_sum[str[2]][1] += str[1]
        cl_sum[str[2]][2] += 1

In [7]:
cl_centers = {} # clusters centers

for el in cl_sum:
    cl_centers[el] = []
    cl_centers[el].append(cl_sum[el][0] * 1.0 / cl_sum[el][2])
    cl_centers[el].append(cl_sum[el][1] * 1.0 / cl_sum[el][2])

In [8]:
import math as m

office_centers = [[33.751277, -118.188740],[25.867736, -80.324116],[51.503016, -0.075479],[52.378894, 4.885084],[39.366487, 117.036146],[-33.868457, 151.205134]]

dist_cl_off = []

for cl in cl_centers:
    for off_ind, off in enumerate(office_centers):
        dist_cl_off.append([m.sqrt((off[0]-cl_centers[cl][0]) ** 2 + (off[1]-cl_centers[cl][1]) ** 2), cl, off_ind])

In [9]:
dist_cl_off.sort()
dist_cl_off[0]

[0.005805158024028328, 348, 5]

In [10]:
cl_centers[348]

[-33.86298195294118, 151.20706369411764]